# RNN 기초 학습, PyTorch로 배우기 ('24.07.31 ~ '24.08.01)

> References : <br>
https://wikidocs.net/60760, https://wikidocs.net/64515

# Recurrent Neural Network

> 대표적인 시퀀스 모델 : 입출력을 시퀀스 단위로 처리하는 모델. 문장은 단어 시퀀스이고, 이들을 처리하기 위해 고안된 모델이다. RNN은 딥러닝에 있어 가장 기본적인 시퀀스 모델임.

- FeedForward NN : 은닉층에서 활성화 함수를 통과한 값이 오직 출력층 방향으로만(한 방향) 향하는 신경망.

- <-> RNN : 은닉층의 노드에서 활성화 함수를 통과한 결과값을 출력층 방향으로도 보내면서, 다시 ***은닉층 노드의 다음 계산의 입력으로 동시에 보내는*** 특징을 갖는다.

- Memory(RNN) cell : 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드이며, 이전의 값을 기억하려 하는 메모리 역할을 수행한다. 각각의 시점에서, 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 output 값을 자신의 입력으로 사용하는 재귀적 활동을 한다.

- Hidden state : 메모리 셀이 출력층 방향 <=> 다음 시점 (t + 1) 셀로 보내는 값을 말한다.

<br>

### 모델의 형태 3가지
- one-to-many : 이미지 캡셔닝 작업 등에 사용될 수 있다. (이미지 한 개 입력 -> 이미지의 제목 문장(단어들) 출력)
- many-to-one : 스팸 메일 분류 작업 등에 사용될 수 있다. (스팸 의심 메일 입력 -> 스팸 메일 여부(T/F) 출력)
- many-to-many : 챗봇, 번역기 등. 입력 문장으로부터 대답 문장을 출력하는 작업에 사용될 수 있다.



## 파이썬으로 RNN 셀 한 층을 구현하기(low-level)

- 실제 PyTorch에서는 (batch_size, timesteps, input_size) 크기의 3D 텐서를 입력으로 받음.

In [91]:
import numpy as np

timesteps = 5 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_size = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다. (d)
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다. (D_h)

inputs = np.random.random((timesteps, input_size)) # 입력에 해당하는 2D 텐서(실제로는 batch size까지 3D로 입력받음)

hidden_state_t = np.zeros((hidden_size, )) # 초기 은닉 상태는 0(벡터)으로 초기화. (8, ) 의 크기를 가짐

print(hidden_state_t)
print(hidden_state_t.size)

[0. 0. 0. 0. 0. 0. 0. 0.]
8


In [92]:
# (8, 4) 크기의 랜덤 값을 갖는 2D 텐서 생성. 입력에 대한 Weight 행렬
W_x = np.random.random((hidden_size, input_size))

# (8, 8) 크기의 랜덤 값을 갖는 2D 텐서 생성. hidden state에 대한 Weight 행렬(정사각행렬)
W_h = np.random.random((hidden_size, hidden_size))

# (8, ) 크기의 랜덤 값을 갖는 1D 텐서 생성. bias(편향)
b = np.random.random((hidden_size, ))

print(np.shape(W_x)) ; print(np.shape(W_h)) ; print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [93]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨
    output_t = np.tanh(np.dot(W_x, input_t) + np.dot(W_h, hidden_state_t) + b)

    total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적

    print(np.shape(total_hidden_states))

    hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0)
print(total_hidden_states) # (timesteps, output_dim)의 크기를 갖는 메모리 셀의 2D 텐서들이 출력됨

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
[[0.94013076 0.95204053 0.99111907 0.99566879 0.97696259 0.97252909
  0.96043119 0.98296719]
 [0.99975433 0.99995622 0.99998792 0.99995452 0.99996576 0.99999221
  0.9998896  0.99986982]
 [0.99972658 0.99988476 0.99998849 0.99991301 0.99996335 0.99999542
  0.99985736 0.99970847]
 [0.99981788 0.99996545 0.99999532 0.99998121 0.99998951 0.99999711
  0.99993062 0.99995727]
 [0.99982683 0.9999509  0.99997696 0.99990008 0.99996616 0.99999029
  0.99989072 0.99976223]]


## PyTorch의 nn.RNN()

- hidden state의 크기인 hidden_size 는 대표적인 RNN의 Hyperparameter로, 사용자가 지정한다.

- timesteps 의미: 시점의 수
    - NLP에서의 의미: 문장의 길이, 즉 문장 내 단어의 개수
    - 예시: 만약 문장이 10개의 단어로 구성되어 있다면, timesteps는 10이 된다.

- input_size 의미: 입력 벡터의 차원
    - NLP에서의 의미: 각 단어 벡터의 차원
    - 예시: One-Hot Encoding 사용 시 어휘의 크기가 5인 경우, 각 단어는 5차원의 원-핫 벡터로 표현되므로 input_size는 5가 된다.

In [94]:
import torch
import torch.nn as nn

batch_size = 1
timesteps = 10
input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

In [95]:
# 입력 텐서 정의 : (batch_size, timesteps, input_size)
inputs = torch.Tensor(batch_size, timesteps, input_size) # 배치 크기는 1, 10번의 시점 동안 5차원의 입력 벡터가 들어가도록 텐서 정의

In [96]:
# RNN cell 정의 : nn.RNN() 메소드 활용
cell = nn.RNN(input_size, hidden_size, batch_first = True)
# batch_first = True : 입력 텐서의 첫 번째 차원이 batch size임을 알려주는 파라미터

In [97]:
# 입력 텐서를 RNN 셀에 입력하여, 출력을 확인한다.(high-level)
outputs, status = cell(inputs)

- RNN 셀은 두 개의 입력을 리턴한다. 첫 번째 리턴값은 모든 시점의 hidden state들이며, "두 번째 리턴값은 마지막 시점"의 hidden state이다.

In [98]:
print(outputs.shape) # 10번의 시점 동안, 8차원의 hidden state가 출력되었다는 의미. 10이라는 숫자는 차원과는 연관 없음.
print(status.shape) # 최종 time-step인 10번째의 hidden state.

torch.Size([1, 10, 8])
torch.Size([1, 1, 8])


### Deep RNN(은닉층 여러 개)

- PyTorch 구현 시, nn.RNN()의 인자인 num_layers 옵션에 값을 부여하여 층을 쌓는 방식으로 구현한다.

> nn.RNN(input_size, hidden_size, num_layers, batch_first)

In [99]:
inputs = torch.Tensor(1, 10, 5)

cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first = True)

outputs, status = cell(inputs)

# 모든 time-step의 hidden_state. 첫 번째 리턴값의 크기는 layer가 한 개인 RNN과 동일. "마지막 층의 모든 시점"에서의 상태.
print(outputs.shape)

# 두 번째 리턴값의 크기는 차이가 있음. [층의 개수, 배치 사이즈, 은닉 상태의 크기]에 해당함.
print(status.shape)

torch.Size([1, 10, 8])
torch.Size([2, 1, 8])


## 양방향 순환 신경망(Bidirectional RNN)

- 시점 t에서의 출력값을 예측할 때, 이전 시점의 데이터(기존) 뿐만 아니라 ***이후 데이터로도 예측할 수 있다***는 아이디어에 기반한다.

- 양방향 RNN은 하나의 출력값을 예측하기 위해 두 개의 메모리 셀을 사용한다.
    - 첫 번째 메모리 셀은 기존과 동일하게 앞 시점의 은닉 상태를 전달받아 현재의 은닉 상태를 계산한다.
    - 두 번째 메모리 셀은 앞 시점의 은닉 상태가 아닌, 뒤 시점의 은닉 상태(Backward states)를 전달받아 현재의 은닉 상태를 계산한다.

In [100]:
inputs = torch.Tensor(1, 10, 5)

cell = nn.RNN(input_size = 5, hidden_size = 8,
              num_layers = 2, batch_first = True,
              bidirectional = True) # Hidden Layer가 2개이고, 양방향 순환 신경망인 경우

outputs, status = cell(inputs)

print(outputs.shape)
print(status.shape)

torch.Size([1, 10, 16])
torch.Size([4, 1, 8])


**해석**

1. outputs.shape : (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2) -> 첫 번째 리턴값의 크기는 단방향 RNN보다 은닉 상태의 크기가 두 배인 16이 됨. "양방향 은닉 상태 값들이 연결됨(concatenate)."

2. status.shape : (은닉층 개수 x 2, 배치 크기, 은닉 상태의 크기) -> {정방향 기준 마지막 시점 + 역방향 기준 첫 번째 시점}이므로, 해당하는 시점(동일)의 출력값을 층의 개수만큼 쌓아 올린 결과값이 된다.

# LSTM(Long Short-Term Memory)

- Vanila(=Simple) RNN의 한계를 극복하기 위한 다양한 RNN의 변형 중 하나이다.

### 바닐라 RNN의 한계 : 장기 의존성 문제
-> Problem of Long-Term Dependencies. 비교적 짧은 시퀀스에 대해서만 효과를 보인다. 바닐라 RNN의 시점(timesteps)이 길어질수록, 앞의 정보가 뒤로 충분히 전달되지 못하는 현상이 발생한다. "앞의 정보가 문장에서 중요"한 역할을 하는 경우 단어 예측을 엉뚱하게 수행할 수 있다.

### LSTM 내부구조
-> LSTM은 은닉층의 메모리 셀에 <입력 게이트, 망각(삭제) 게이트, 출력 게이트> 를 추가하여, 불필요한 기억들을 지우고 기억해야 할 것들을 선별한다. + cell state(셀 상태)라는 값을 추가하여 은닉 사태를 계산하는 식이 바닐라 RNN보다 복잡해지지만, 긴 시퀀스의 입력 처리 시 개선된 성능을 보인다.

- 셀 상태 : 이전 시점의 셀 상태가 다음 시점의 셀 상태를 구하기 위한 입력으로서 사용된다.

- 입력 게이트, 망각 게이트, 출력 게이트에는 공통적으로 sigmoid가 존재하여, 0 ~ 1의 값으로 (망각/기억) 게이트 출력을 조절한다.

- 망각 게이트는 이전 시점의 입력을 얼마나 반영할지 결정하고, 입력 게이트는 현재 시점의 입력을 얼마나 반영할지를 결정하는 역할을 수행한다.

### PyTorch의 nn.LSTM()
-> nn.LSTM(input_dim, hidden_size, batch_first) # RNN 셀과 동일한 파라미터

In [101]:
# LSTM 사용 예시
inputs = torch.Tensor(1, 10, 5)

cell = nn.LSTM(input_size = 5, hidden_size = 8,
              num_layers = 2, batch_first = True,
              bidirectional = True) # LSTM Layer가 2개이고, 양방향 LSTM인 경우

outputs, (hidden_state, cell_state) = cell(inputs)

print(outputs.shape)
print(hidden_state.shape)
print(cell_state.shape)

torch.Size([1, 10, 16])
torch.Size([4, 1, 8])
torch.Size([4, 1, 8])


**해석**

1. outputs : 각 시점에 대한 모든 레이어의 출력이다. 양방향 LSTM이기 떄문에, 각 시점에서 순방향과 역방향의 출력을 합친 8 + 8 = 16차원의 벡터가 된다.

2. hidden_state : (num_layer x 2(양방향) : 4, batch_size : 1, hidden_size : 8)

3. cell_state : hidden_state와 상동

차별점 : LSTM이므로 두 번째 인자로 Vanila RNN와 달리 <br> status -> (hidden_state, cell_state)의 tuple을 리턴받음.

# GRU(Gated Recurrent Unit, 게이트 순환 유닛)
-> LSTM의 여전한 장기 의존성 문제에 대한 해결책을 유지하면서, 은닉 상태를 업데이트하는 계산을 간소화하였다.

- GRU와 LSTM 중 성능 측면에서 더 나은 것을 단정지을 수는 없다. LSTM으로 최적의 하이퍼파라미터를 찾아낸 상황이라면 굳이 바꿔 사용할 필요는 없다.

- 경험적으로 데이터 양이 소량일 때는 매개변수의 양이 적은 GRU가 조금 더 낫고, 데이터 양이 많을 경우 LSTM이 더 낫다고 본다고 한다.

### PyTorch의 nn.GRU()
-> nn.GRU(input_dim, hidden_size, batch_first) # RNN 셀과 동일한 파라미터

In [102]:
# LSTM 사용 예시
inputs = torch.Tensor(1, 10, 5)

cell = nn.GRU(input_size = 5, hidden_size = 8,
              num_layers = 2, batch_first = True,
              bidirectional = True) # GRU Layer가 2개이고, 양방향 GRU인 경우

outputs, cell_state = cell(inputs) # GRU는 hidden state만 리턴한다. (LSTM 간소화)

print(outputs.shape)
print(cell_state.shape)

torch.Size([1, 10, 16])
torch.Size([4, 1, 8])


# 문자 단위 RNN(Char RNN)
-> RNN의 입출력의 단위가 단어 레벨이 아니라, 문자 레벨로 하여 RNN을 구현한 것을 말한다.

---

## I. 문자 시퀀스 apple 입력 -> pple!을 출력하는 RNN 구현

### 1. 훈련 데이터 전처리

In [103]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [104]:
input_str = 'apple'
label_str = 'pple!'

# set으로 중복되지 않게 string을 char로 분리 -> list에 문자들 담기 -> 사전순 오름차순 정렬
char_vocab = sorted(list(set(input_str + label_str)))

In [105]:
# 하이퍼파라미터 정의 : 입력이 One-Hot Vector이므로 입력의 크기는 문자 집합의 크기와 동일해야 한다.
vocab_size = len(char_vocab)
input_size = vocab_size

hidden_size = 5
output_size = 5

learning_rate = 0.1

In [106]:
# 문자 집합에 고유한 정수를 부여 : 레이블링
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 리스트 원소 -> 딕셔너리화
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [107]:
# 예측 결과를 다시 문자 시퀀스로 보기 위해서, 레이블된 정수로부터 문자를 얻을 수 있게 함
index_to_char = {}

for key, value in char_to_index.items():
    index_to_char[value] = key # key, value 바꿔주면 끝
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [108]:
x_data = [char_to_index[c] for c in input_str] # list
y_data = [char_to_index[c] for c in label_str] # list
print(x_data) # a, p, p, l, e
print(y_data) # p, p, l, e, !

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


- PyTorch의 nn.RNN()은 3차원 텐서를 입력받는다. 배치 차원 추가가 필요함.

In [109]:
x_data = torch.tensor(x_data).unsqueeze(0) # x_data = [x_data]
y_data = torch.tensor(y_data).unsqueeze(0) # y_data = [y_data]
print(x_data)
print(y_data)

tensor([[1, 4, 4, 3, 2]])
tensor([[4, 4, 3, 2, 0]])


- x_data : 1(a), 4(p), 4(p), 3(l), 2(e)
- vocab_size : 5(!, a, e, l, p 5개)
- np.eye : NumPy 단위행렬 생성 메소드

In [110]:
# 입력 시퀀스의 각 문자들을 One-Hot Vector화
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [111]:
# 입력, 레이블 데이터 텐서화
x_one_hot = np.array(x_one_hot)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


### 2. 모델 구현하기

In [112]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first = True) # RNN 셀 구현
        self.fc = nn.Linear(hidden_size, output_size, bias = True) # 출력층(Fully Connected)

    def forward(self, x): # 구현한 RNN 셀과 출력층 연결
        x, status = self.rnn(x)
        x = self.fc(x)
        return x

In [113]:
net = Net(input_size, hidden_size, output_size)

outputs = net(X)
print(outputs.shape) # 3D 텐서, [배치 차원, "시점(timesteps)", 출력의 크기]
print(Y.shape)

torch.Size([1, 5, 5])
torch.Size([1, 5])


In [114]:
# 2D 텐서로 변환, view를 사용하여 배치 차원과 시점 차원을 하나로 만든다. 정확도(accuracy) 측정 시 사용됨.
print(outputs.view(-1, input_size).shape)

print(Y.view(-1).shape)

torch.Size([5, 5])
torch.Size([5])


In [115]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [116]:
EPOCH = 50

for epoch in range(EPOCH + 1):
    optimizer.zero_grad()

    outputs = net(X)

    # (prediction, 정답 Y) : view를 하는 이유는 Batch 차원 제거를 위함
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()

    optimizer.step() # 파라미터 업데이트

    # - - - 모델 예측 확인 코드 - - -
    result = outputs.data.numpy().argmax(axis = 2) # 최종 예측값인 각 timestep 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    if epoch % 5 == 0:
        print("epoch", epoch, "/ loss :", loss.item(), "/ prediction :", result,
              "/ true Y :", y_data, "/ prediction str :", result_str)

epoch 0 / loss : 1.8081724643707275 / prediction : [[0 0 0 0 0]] / true Y : tensor([[4, 4, 3, 2, 0]]) / prediction str : !!!!!
epoch 5 / loss : 0.8989709615707397 / prediction : [[4 4 4 4 0]] / true Y : tensor([[4, 4, 3, 2, 0]]) / prediction str : pppp!
epoch 10 / loss : 0.3196743130683899 / prediction : [[4 4 3 2 0]] / true Y : tensor([[4, 4, 3, 2, 0]]) / prediction str : pple!
epoch 15 / loss : 0.06679805368185043 / prediction : [[4 4 3 2 0]] / true Y : tensor([[4, 4, 3, 2, 0]]) / prediction str : pple!
epoch 20 / loss : 0.01642611250281334 / prediction : [[4 4 3 2 0]] / true Y : tensor([[4, 4, 3, 2, 0]]) / prediction str : pple!
epoch 25 / loss : 0.0056962063536047935 / prediction : [[4 4 3 2 0]] / true Y : tensor([[4, 4, 3, 2, 0]]) / prediction str : pple!
epoch 30 / loss : 0.0028663554694503546 / prediction : [[4 4 3 2 0]] / true Y : tensor([[4, 4, 3, 2, 0]]) / prediction str : pple!
epoch 35 / loss : 0.0017348633846268058 / prediction : [[4 4 3 2 0]] / true Y : tensor([[4, 4, 3, 

## II. 더 많은 데이터 문자 단위 RNN 구현

### 1. 훈련 데이터 전처리

In [181]:
import torch
import torch.nn as nn
import torch.optim as optim

In [182]:
# 긴 문장의 데이터 예제
sentence = ("Success is not final, failure is not fatal: It is the courage to continue that counts. In the face of adversity, the true measure of a person is not how they fall, but how they rise after falling.")

In [183]:
# 문자 집합 생성, 각 문자에 고유한 정수 부여
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c : i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩
print(char_dic) # 공백 또한 하나의 원소로 취급한다. (' ': 6)

dic_size = len(char_dic)
print("문자 집합의 크기 :", dic_size)

{'g': 0, 'u': 1, 'n': 2, '.': 3, ',': 4, 'b': 5, ' ': 6, 'c': 7, 'S': 8, ':': 9, 'a': 10, 'v': 11, 'w': 12, 'f': 13, 'I': 14, 'p': 15, 'r': 16, 'h': 17, 'y': 18, 's': 19, 'i': 20, 'l': 21, 'e': 22, 'o': 23, 'm': 24, 't': 25, 'd': 26}
문자 집합의 크기 : 27


In [184]:
# 하이퍼파라미터 설정
hidden_size = dic_size # 은닉 상태의 크기를 입력의 크기와 동일하게 설정, 사용자 선택으로 다른 값을 줘도 됨.
sequence_length = 10 # 앞서 만든 샘플을 10개 단위로 끊음.
learning_rate = 0.1

In [185]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i : i + sequence_length]
    y_str = sentence[i + 1 : i + sequence_length + 1]

    if (i >= 0 and i <= 5) or (i >= len(sentence) - sequence_length - 5 and i <= len(sentence) - sequence_length):
        print(i, x_str, '->', y_str)
        print()
        if (i == 5):
            print('. . . 중략 . . .')
            print()

    x_data.append([char_dic[c] for c in x_str]) # 각각의 정수 레이블로 저장(샘플의 각 문자들은 고유한 정수로 인코딩됨)
    y_data.append([char_dic[c] for c in y_str]) # 마찬가지

print(x_data[0]) # if you wan -> [18, 12, 6, 16, 21, 1, 6, 11, 10, 2]
print(y_data[0]) # f you want -> [12, 6, 16, 21, 1, 6, 11, 10, 2, 23]

0 Success is -> uccess is 

1 uccess is  -> ccess is n

2 ccess is n -> cess is no

3 cess is no -> ess is not

4 ess is not -> ss is not 

5 ss is not  -> s is not f

. . . 중략 . . .

181  after fal -> after fall

182 after fall -> fter falli

183 fter falli -> ter fallin

184 ter fallin -> er falling

185 er falling -> r falling.

[8, 1, 7, 7, 22, 19, 19, 6, 20, 19]
[1, 7, 7, 22, 19, 19, 6, 20, 19, 6]


In [186]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x_data를 One-Hot Encoding

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print('훈련 데이터의 크기 : {}'.format(X.shape)) # [170 : len(sentence), 10 : sequence_length, 25 : dic_size]
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([186, 10, 27])
레이블의 크기 : torch.Size([186, 10])


### 2. 모델 구현하기

- 은닉층을 두 개 이용함.

In [187]:
class Net2(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # hidden_size == dic_size
        super(Net2, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias = True)

    def forward(self, x):
        x, status = self.rnn(x)
        x = self.fc(x)
        return x

In [196]:
net2 = Net2(dic_size, hidden_size, 2) # num_layers = layers = 2. 은닉층 두 개 이용

In [197]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net2.parameters(), learning_rate)

In [198]:
outputs = net2(X)
print(outputs.shape)
print(outputs.view(-1, dic_size).shape) # 2D 텐서로 변환
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([186, 10, 27])
torch.Size([1860, 27])
torch.Size([186, 10])
torch.Size([1860])


In [199]:
EPOCH = 50

for epoch in range(EPOCH + 1):
    optimizer.zero_grad()
    outputs = net2(X) # (170, 10, 25) 크기를 가진 텐서를 매 epoch 마다 모델의 입력으로 사용

    # (prediction, 정답 Y) : view를 하는 이유는 Batch 차원 제거를 위함
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim = 2) # 예측된 값 outputs에서 가장 높은 값을 갖는 인덱스를 찾아 저장
    predict_str = ""

    for j, result in enumerate(results):
        if j == 0: # 처음에는 모든 예측 결과를 문자열로 변환, predict_str에 추가
            predict_str += ''.join([char_set[t] for t in result])

        else: # 그 다음에는 마지막 글자만 반복 추가한다.
            predict_str += char_set[result[-1]]
    if epoch % 5 == 0:
        print(f"epoch : {epoch} / model's predicted string results : {predict_str}")

epoch : 0 / model's predicted string results : pfnnsptptppptsp,pptp,p,pppptsptppptsp,ppppfpfpptppfpspttpttstptppntptpsptptpsppntpptps,,pptpsp,pptpt,ppptsssttp,ptpsptptpppppptftptpppptpfttpptppptsp,psptpsp,npppppfptpspsptpsp,,tptptpppfp,ppppps
epoch : 5 / model's predicted string results :  ly   lly    ll  y y lyy l  y     y iy y y usy y  yl    y  l  y       y ly l    l  ylyIu l  y   y  iy ly l  l iyi i  yiy  y  y ly ly yI  l      yyl  o   y   y  lpsly  l  y y   yi   y t   lsly lyy
epoch : 10 / model's predicted string results :  t t   fo t t   t te f etfot    t t t t te fe    f   eot tfe e t ee etet e t ht  oe  ef  t t e tfhtt t  t   e e t t e t      etfht  teeo   eo   fht ttf t    f tfe t   eh       fht tfet ef  teette
epoch : 15 / model's predicted string results : e t    eo l t to slu tetslot t ue t to  le  u t  n t tot t u t t tn t uelt tfheot t ulfu t t tole tf tlue t t t t t t t  fu  uel tf tlue t t to f t tst t t tf tfu tu  tut t t tfu  lel llfeoleu  t
epoch : 20 / model's predi